# SAR Calibration Tool: Tutorial

The current Jupyter Notebook can be used to exploit the SAR Calibration Tool functionalities to analyse a SAR product acquired over a calibration site and store the results in the calibration database.

## Installation

In order to install the SAR Calibration Tool follow the instruction reported in the documentation.

## Setup

Import the required packages.

In [ ]:
import os

from sct.analysis.calibrationsite import CalibrationSite
from sct.configuration.configuration import Configuration

Define useful inputs:

- Configuration file, defining the parameters to be used during processing

  Note: Remember to properly update it before running the tool. In particular, properly set the paths (to auxiliary files, ...) according to your local setup. By default, the configuration file is placed in the ``resources/configuration`` sub-folder of the SCT distribution).


In [ ]:
configuration_file       = os.path.join('../../resources/configuration/configuration_file.xml')

- Calibration site parameters, i.e.:
  - its identifier
  - the calibration targets file, describing position, validity, ... of each target belonging to the site
  - the bounding polygon file, describing the area covered by the site

  Note: By default, these file are placed in the ``resources/calibration_db/<calibration site identifier>`` sub-folder of the SCT distribution. Sample files are provided within it.

In [ ]:
calibration_site_id      = 'surat_basin'
calibration_site_dir     = os.path.join('../../resources/calibration_sites/', calibration_site_id)
calibration_targets_file = os.path.join(calibration_site_dir, 'calibration_targets.xlsx')
bounding_polygon_file    = os.path.join(calibration_site_dir, 'bounding_polygon.geojson')

- Calibration database file, including the results collected so far and where to store the new results. The same file can be used if you want to append the new results.

  Note: By default, these file are placed in the ``resources/calibration_sites/<calibration site identifier>`` sub-folder of the SCT distribution. Sample files are provided within it.

In [ ]:
calibration_db_dir       = os.path.join('../../resources/calibration_db/', calibration_site_id)
calibration_db_file_in   = os.path.join(calibration_db_dir, 'calibration_db.xlsx')
calibration_db_file_out  = os.path.join(calibration_db_dir, 'calibration_db__updated.xlsx')

- SAR product path, indicating the product to be analysed

  Note: By default, SAR products are placed in the ``data/sar_products`` sub-folder of the SCT distribution.

In [ ]:
sar_products_dir         = '../../data/sar_products'
sar_product_dir          = os.path.join(sar_products_dir, 'S1A_IW_SLC__1SSH_20201204T083311_20201204T083338_035533_042794_32B3.SAFE')

- Auxiliary files:
  - (optionally) Orbit file path, indicating the external orbit file to be used during processing
  - Ionosphere maps folder, indicating the location of maps to be used to apply ionospheric delay compensation
  - Troposphere maps folder, indicating the location of maps to be used to apply tropospheric delay compensation. In this folder shall be present also the spherical harmonics coefficients files and then grid point coordinates files, by default included in the SCT distribution
  
Note: Ionosphere and troposphere maps folders shall be specified in the SCT configuration file. By default, auxiliary files are placed in the ``data/auxiliary_files/*`` sub-folders of the SCT distribution.

In [ ]:
orbits_dir               = '../../data/auxiliary_files/orbits'
orbit_file               = os.path.join(orbits_dir, 'S1A_OPER_AUX_RESORB_OPOD_20201204T112515_V20201204T070214_20201204T101944.EOF')

### SAR products and auxiliary files retrieval

Note that both SAR products and auxiliary files shall be manually downloaded and made available to SCT as above described.

SAR products and auxiliary orbit files can be downloaded from Copernicus Open Access Hub (https://scihub.copernicus.eu/).

Ionosphere and troposphere maps can be downloaded respectively from:

- Ionosphere maps (cod* files): https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/atmospheric_products.html
- Troposphere maps (VMF3* files): https://vmf.geo.tuwien.ac.at/

In case of zipped archives, be sure to properly unzip them.

## Run SAR Calibration Tool

Load configuration file:

In [ ]:
configuration = Configuration(configuration_file)

Initialise and load calibration site:

In [ ]:
cs = CalibrationSite(calibration_site_id, calibration_targets_file, bounding_polygon_file)

cs.load_calibration_db(calibration_db_file_in)

Add a new overpass (i.e. a new SAR product to be analysed):

In [ ]:
sar_product = cs.add_overpass(sar_product_dir)

#sar_product = cs.add_overpass(sar_product_dir, orbit_file)   # use this in case you want to use an external orbit for processing

Analyse the SAR product:

In [ ]:
results = cs.analyse_overpass(sar_product, configuration.calibration_site_analysis_conf)

Store results in the calibration database:

In [ ]:
cs.save_calibration_db(calibration_db_file_out)

In order to get more details on performed processing, inspect log file saved on disk. Note that log file contains also debug messages, in addition to information messages printed to stdout/stderr.

In order to display results, follow the instructions reported in the sct_display_results Jupyter Notebook.